# Data Clean Up

## Libraries

In [7]:
import pandas as pd
import pandas_datareader.data as web
import yfinance as yf
import datetime
import numpy as np
import xlwings as xw
from pathlib import Path
import fred_functions

## ETF Daily Returns

### Clean Up

###### import ETF file

In [15]:
df_returns = pd.read_excel('../df_returns.xlsx',index_col=0,sheet_name='Sheet1')

###### Select "SPY" and "VBMFX". SPY inception was in 1993 while VBMFX was started in 1986. The data set will cover dates from February 1, 1993 to September 30, 2025

In [ ]:
df = df_returns[['SPY','VBMFX']]
df = df.loc['1993-02-01':'2025-09-30']

,SPY,VBMFX
Date,,
1993-02-01,24.485952,2.423037
1993-02-02,24.537830,2.418192
1993-02-03,24.797216,2.420614
1993-02-04,24.900978,2.425460
1993-02-05,24.883671,2.432732
...,...,...
2025-09-24,661.099976,9.770000
2025-09-25,658.049988,9.760000
2025-09-26,661.820007,9.760000
